## Gaussian Pyramid Levels

In [2]:
%load_ext autoreload
%autoreload 2

In [ ]:
!pip install online_triplet_loss

from online_triplet_loss.losses import *
import numpy as np
import cv2
import matplotlib.pyplot as plt
from data import get_triplets
from PIL import Image
import random
import os

import torch
import torch.nn as nn
from torchvision import datasets, transforms
import torchvision.transforms.functional as F
from torch.utils.data import DataLoader


## Learning

In [33]:
class Backbone(nn.Module):
    def __init__(self, in_chan, out_dim):
        super(Backbone, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=in_chan, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.maxpool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.fc = nn.Linear(in_features=1568, out_features=out_dim)
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.maxpool1(x)
        
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.maxpool2(x)
        
        x = x.view(x.size(0), -1)  
        
        x = self.fc(x)
        
        return x

In [28]:
transform = transforms.Compose([
    transforms.Resize((28, 28)),
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

In [29]:
def get_embeddings(flist, model):        
    sample = random.sample(flist, 256)
    
    inp = []
    for f in sample:
        inp.append(transform(Image.open(f)))
    
    out = model(torch.stack(inp, dim=0))

    return out, sample

In [30]:
def get_labels(flist):
    
    labels = []
    for f in flist:
        # get rid of the dir name
        fname = os.path.split(f)[1]
        
        # get rid of the augmentor ID
        fname = fname.split(".jpg")[0]
        
        label = int(fname.split("_")[2])
        labels.append(label)
        
    return np.array(labels)

In [31]:
# use the augmented images
src = "pet/train/output"
flist = [os.path.join(src, f) for f in sorted(os.listdir(src))]


In [ ]:
import torch.optim as optim

model = Backbone(in_chan=3, out_dim=512)

optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10000

model.train()
    
for epoch in range(num_epochs):
    
    # Zero the gradients
    optimizer.zero_grad()
    
    embed, fbatch = get_embeddings(flist, model)
    labels = get_labels(fbatch)
    
    ### TODO? Make margin adaptive?
    loss, _ = batch_all_triplet_loss(torch.tensor(labels), embed, margin=5)

    print(f"Epoch: [{epoch+1}/{num_epochs}]\tLoss: {loss.item():.6f} Positive frac: {_.item():.6f}")

    # Backward pass - compute gradients and update weights
    loss.backward()
    optimizer.step()

Epoch: [1/10000]	Loss: 4.727259 Positive frac: 1.000000
Epoch: [2/10000]	Loss: 3.269098 Positive frac: 1.000000
Epoch: [3/10000]	Loss: 3.251437 Positive frac: 0.957818
Epoch: [4/10000]	Loss: 3.150373 Positive frac: 0.807743
Epoch: [5/10000]	Loss: 2.316380 Positive frac: 0.671260
Epoch: [6/10000]	Loss: 3.278442 Positive frac: 0.605643
Epoch: [7/10000]	Loss: 1.518164 Positive frac: 0.124016
Epoch: [8/10000]	Loss: 2.263373 Positive frac: 0.335302
Epoch: [9/10000]	Loss: 3.561862 Positive frac: 0.484252
Epoch: [10/10000]	Loss: 1.971492 Positive frac: 0.130906
Epoch: [11/10000]	Loss: 2.486673 Positive frac: 0.135039
Epoch: [12/10000]	Loss: 3.863686 Positive frac: 0.148622
Epoch: [13/10000]	Loss: 2.853461 Positive frac: 0.137795
Epoch: [14/10000]	Loss: 4.561041 Positive frac: 0.151575
Epoch: [15/10000]	Loss: 3.394362 Positive frac: 0.091207
Epoch: [16/10000]	Loss: 2.954612 Positive frac: 0.052362
Epoch: [17/10000]	Loss: 4.279675 Positive frac: 0.041010
Epoch: [18/10000]	Loss: 2.240755 Positiv